In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import json
from random import randint
from time import sleep
from selenium import webdriver
import os
import datetime

In [2]:
# Get current 30 Year fixed rate as of today
url = 'https://www.mortgagenewsdaily.com/mortgage-rates/30-year-fixed'
html = requests.get(url)
soup = bs(html.text, 'html.parser')
soup
mortgageRate = soup.find('div', {'class': 'price'}).text
rate = float(mortgageRate[0:4])
mortgageRate = rate/100
mortgageRate

0.0602

In [9]:
rentalData = pd.read_csv('sdRents.csv')
homePriceData = pd.read_csv('redfin_2022-07-14-23-37-58.csv')

In [10]:
bed = 1
bath = 1

                                     

b_b = rentalData[(rentalData['Bedrooms'] == bed) & (rentalData['Bathrooms'] == bath)]
benchmarkRentalRate = int(b_b['Price'].mean())
HOA = int(homePriceData['HOA/MONTH'].mean())


In [11]:
## Updating function to get home price to payment

def paymentCalculator(homePrice, bed, bath, downPaymentPercentage, closingCostPercentage, loanPayments, propertyTaxRate, 
                      HOA, mortgageInsuranceRate, mortgageRate, areaRentalRate):
    downPayment = homePrice*downPaymentPercentage
    closingCosts = (homePrice-downPayment)*closingCostPercentage
    Principal = (homePrice-downPayment)
    propertyTax = (propertyTaxRate*homePrice)/12
    monthlyMortgageRate = mortgageRate/12
    monthlyMortgageRate_1 = monthlyMortgageRate+1 
    loanPayments_1 = loanPayments-1
    P_I = Principal*monthlyMortgageRate*monthlyMortgageRate_1**loanPayments/monthlyMortgageRate_1**loanPayments_1
    mortgagePayment = P_I+propertyTax+HOA
    if downPaymentPercentage <.20:
        mortgageInsurance = (Principal * mortgageInsuranceRate)/12
    else: 
        mortgageInsurance = 0
    mortgagePayment = P_I+propertyTax+HOA+mortgageInsurance
    yearlyCost = mortgagePayment*12
    investmentCosts = {'HomePrice':[homePrice], 'bed':[bed], 'bath':[bath],'downPayment':[downPayment], 'closingCosts':[closingCosts], 
                   'mortgageRate':[mortgageRate], 'loanPayments':[loanPayments], 'Principal':[Principal], 
                   'propertyTax':[propertyTax], 'HOA':[HOA], 'P&I':[P_I], 'mortgageInsurance':[mortgageInsurance],
                       'mortgagePayment':[mortgagePayment], 'yearlyCost':[yearlyCost],'benchmarkRate':[areaRentalRate]}
    global df
    df = pd.DataFrame(data=investmentCosts)
    a = int(df['HomePrice']/df['mortgagePayment'])
    if (mortgagePayment-areaRentalRate)>0:
        print('Will not cash flow by $', (mortgagePayment-areaRentalRate))
        print("$", a, "is the amount of home price per dollar of payment")
        print((areaRentalRate*a), "is area rental rate home value break even")
    else:
        print('Will cash flow $', ((mortgagePayment-areaRentalRate)*-1), 'monthly')
        print('Will cash flow $', (((mortgagePayment-areaRentalRate)*12)*-1), 'yearly')
        print("$", a, "is the amount of home price per rental dollar of mortgage payment")
        print((areaRentalRate * a), "is area rental rate home value break even")
    return df

In [12]:
paymentCalculator(380000,bed,bath,.20,.03,360,.0125,HOA,.02,mortgageRate, benchmarkRentalRate)

Will not cash flow by $ 64.55075111111091
$ 164 is the amount of home price per dollar of payment
367852 is area rental rate home value break even


,HomePrice,bed,bath,downPayment,closingCosts,mortgageRate,loanPayments,Principal,propertyTax,HOA,P&I,mortgageInsurance,mortgagePayment,yearlyCost,benchmarkRate
0,380000,1,1,76000.0,9120.0,0.0602,360,304000.0,395.833333,379,1532.717418,0,2307.550751,27690.609013,2243


In [14]:
def paymentCalculator(homePrice, bed, bath, downPaymentPercentage, closingCostPercentage, loanPayments, propertyTaxRate, 
                      HOA, mortgageInsuranceRate, mortgageRate, areaRentalRate):
    downPayment = homePrice*downPaymentPercentage
    closingCosts = (homePrice-downPayment)*closingCostPercentage
    Principal = (homePrice-downPayment)
    propertyTax = (propertyTaxRate*homePrice)/12
    monthlyMortgageRate = mortgageRate/12
    monthlyMortgageRate_1 = monthlyMortgageRate+1 
    loanPayments_1 = loanPayments-1
    P_I = Principal*monthlyMortgageRate*monthlyMortgageRate_1**loanPayments/monthlyMortgageRate_1**loanPayments_1
    mortgagePayment = P_I+propertyTax+HOA
    if downPaymentPercentage <.20:
        mortgageInsurance = (Principal * mortgageInsuranceRate)/12
    else: 
        mortgageInsurance = 0
    mortgagePayment = P_I+propertyTax+HOA+mortgageInsurance
    yearlyCost = mortgagePayment*12
    investmentCosts = {'HomePrice':[homePrice], 'bed':[bed], 'bath':[bath],'downPayment':[downPayment], 'closingCosts':[closingCosts], 
                   'mortgageRate':[mortgageRate], 'loanPayments':[loanPayments], 'Principal':[Principal], 
                   'propertyTax':[propertyTax], 'HOA':[HOA], 'P&I':[P_I], 'mortgageInsurance':[mortgageInsurance],
                       'mortgagePayment':[mortgagePayment], 'yearlyCost':[yearlyCost],'benchmarkRate':[areaRentalRate]}
    global df
    df = pd.DataFrame(data=investmentCosts)
    a = int(df['HomePrice']/df['mortgagePayment'])
    if (mortgagePayment-areaRentalRate)>0:
        print('Will not cash flow by $', (mortgagePayment-areaRentalRate))
        print("$", a, "is the amount of home price per dollar of payment")
        print((areaRentalRate*a), "is area rental rate home value break even")
    else:
        print('Will cash flow $', ((mortgagePayment-areaRentalRate)*-1), 'monthly')
        print('Will cash flow $', (((mortgagePayment-areaRentalRate)*12)*-1), 'yearly')
        print("$", a, "is the amount of home price per rental dollar of mortgage payment")
        print((areaRentalRate * a), "is area rental rate home value break even")
    return df

paymentCalculator(1000000,2,1,.20,.03,360,.0125,408,.02,mortgageRate, benchmarkRentalRate)

Will not cash flow by $ 3240.133555555555
$ 182 is the amount of home price per dollar of payment
408226 is area rental rate home value break even


,HomePrice,bed,bath,downPayment,closingCosts,mortgageRate,loanPayments,Principal,propertyTax,HOA,P&I,mortgageInsurance,mortgagePayment,yearlyCost,benchmarkRate
0,1000000,2,1,200000.0,24000.0,0.0602,360,800000.0,1041.666667,408,4033.466889,0,5483.133556,65797.602667,2243
